In [2]:
import gc
import os
from datetime import datetime
import pickle
import psutil
from tqdm import tqdm, tqdm_notebook
from functools import partial, update_wrapper
from itertools import product
from multiprocessing import Pool

import bloscpack as bp

import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

from YSMLT import utils as g_utils
from YSMLT.series import utils as ts_utils

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy.signal import welch, find_peaks
from scipy import stats
from scipy.special import entr
from scipy.stats import entropy
from scipy.stats import percentileofscore
from tsfresh.feature_extraction import feature_calculators

### mean -0.31079195780498403
### std  2.4983141023158186

In [3]:
pdf_trn = pd.read_csv('../input/train_clean.csv')
pdf_tst = pd.read_csv('../input/test_clean.csv')

with open('../input/batch_ids_trn.pkl', 'rb') as f:
    batch_id_trn = pickle.load(f)
with open('../input/batch_ids_tst.pkl', 'rb') as f:
    batch_id_tst = pickle.load(f)
    
with open('../input/target_codes.pkl', 'rb') as f:
    encoder = pickle.load(f)
    


In [4]:
make_srs_feats(pdf_trn, batch_id_trn, step=1000, wndw=1000, tag='final')

100%|██████████| 10/10 [00:04<00:00,  2.45it/s]


In [5]:
make_srs_feats(pdf_tst, batch_id_tst, step=1000, wndw=1000, mode='test', tag='final')

100%|██████████| 20/20 [00:01<00:00, 13.17it/s]


In [3]:
def make_srs_feats(raw_pdf, batch_ids, step, wndw, mode='train', tag='basicwithnew3'):
    for i in tqdm(range(0, len(batch_ids))):
        sgnl_ndcs = batch_ids[i]
        
        if mode == 'train':
            trgt = raw_pdf['open_channels'].iloc[sgnl_ndcs]

        # ------------------------------------------------
        sgnl_p1 = raw_pdf['signal'].iloc[sgnl_ndcs]
        p1_fill = sgnl_p1.mean()

#         sgnl_p1bx = sgnl_p1.shift(-10, fill_value=p1_fill)
#         sgnl_p1b9 = sgnl_p1.shift(-9, fill_value=p1_fill)
#         sgnl_p1b8 = sgnl_p1.shift(-8, fill_value=p1_fill)
#         sgnl_p1b7 = sgnl_p1.shift(-7, fill_value=p1_fill)
#         sgnl_p1b6 = sgnl_p1.shift(-6, fill_value=p1_fill)
#         sgnl_p1b5 = sgnl_p1.shift(-5, fill_value=p1_fill)
#         sgnl_p1b4 = sgnl_p1.shift(-4, fill_value=p1_fill)
        sgnl_p1b3 = sgnl_p1.shift(-3, fill_value=p1_fill)
        sgnl_p1b2 = sgnl_p1.shift(-2, fill_value=p1_fill)
        sgnl_p1b1 = sgnl_p1.shift(-1, fill_value=p1_fill)
        sgnl_p1f1 = sgnl_p1.shift(1, fill_value=p1_fill)
        sgnl_p1f2 = sgnl_p1.shift(2, fill_value=p1_fill)
        sgnl_p1f3 = sgnl_p1.shift(3, fill_value=p1_fill)
#         sgnl_p1f4 = sgnl_p1.shift(4, fill_value=p1_fill)
#         sgnl_p1f5 = sgnl_p1.shift(5, fill_value=p1_fill)
#         sgnl_p1f6 = sgnl_p1.shift(6, fill_value=p1_fill)
#         sgnl_p1f7 = sgnl_p1.shift(7, fill_value=p1_fill)
#         sgnl_p1f8 = sgnl_p1.shift(8, fill_value=p1_fill)
#         sgnl_p1f9 = sgnl_p1.shift(9, fill_value=p1_fill)
#         sgnl_p1fx = sgnl_p1.shift(10, fill_value=p1_fill)

#         sgnl_diff_p1b5 = sgnl_p1b5-sgnl_p1
#         sgnl_diff_p1b4 = sgnl_p1b4-sgnl_p1
        sgnl_diff_p1b3 = sgnl_p1b3-sgnl_p1
        sgnl_diff_p1b2 = sgnl_p1b2-sgnl_p1
        sgnl_diff_p1b1 = sgnl_p1b1-sgnl_p1
#         sgnl_diff_p1f5 = sgnl_p1f5-sgnl_p1
#         sgnl_diff_p1f4 = sgnl_p1f4-sgnl_p1
        sgnl_diff_p1f3 = sgnl_p1f3-sgnl_p1
        sgnl_diff_p1f2 = sgnl_p1f2-sgnl_p1
        sgnl_diff_p1f1 = sgnl_p1f1-sgnl_p1

#         sgnl_grad = pd.Series(np.gradient(sgnl_p1))
        
#         sgnl_exp = pd.Series(np.exp(sgnl_p1))

#         sgnl_diff_p1b5_p1f5 = sgnl_p1b5 - sgnl_p1f5
        sgnl_diff_p1b3_p1f3 = sgnl_p1b3 - sgnl_p1f3
        sgnl_diff_p1b1_p1f1 = sgnl_p1b1 - sgnl_p1f1

#         sgnl_stck0 = pd.concat(
#             [
#                 sgnl_p1bx, sgnl_p1b9, sgnl_p1b8, sgnl_p1b7, sgnl_p1b6, 
#                 sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, 
#                 sgnl_p1, 
#                 sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5,
#                 sgnl_p1f6, sgnl_p1f7, sgnl_p1f8, sgnl_p1f9, sgnl_p1fx,
#             ], axis=1
#         )
#         sgnl_stckf = pd.concat(
#             [
#                 sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5,
#                 sgnl_p1f6, sgnl_p1f7, sgnl_p1f8, sgnl_p1f9, sgnl_p1fx,
#             ], axis=1
#         )
#         sgnl_stckb = pd.concat(
#             [
#                 sgnl_p1bx, sgnl_p1b9, sgnl_p1b8, sgnl_p1b7, sgnl_p1b6, 
#                 sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1
#             ], axis=1
#         )
        
#         sgnl_intgrl0 = pd.Series(np.array([np.trapz(sub) for sub in sgnl_stck0.values]))
#         sgnl_intgrlf = pd.Series(np.array([np.trapz(sub) for sub in sgnl_stckf.values]))
#         sgnl_intgrlb = pd.Series(np.array([np.trapz(sub) for sub in sgnl_stckb.values]))

#         sgnl_f_avg = sgnl_stckf.mean(1)
#         sgnl_f_q10 = sgnl_stckf.quantile(0.10, axis=1)
#         sgnl_f_q25 = sgnl_stckf.quantile(0.25, axis=1)
#         sgnl_f_q50 = sgnl_stckf.quantile(0.50, axis=1)
#         sgnl_f_q75 = sgnl_stckf.quantile(0.75, axis=1)
#         sgnl_f_q90 = sgnl_stckf.quantile(0.90, axis=1)
#         sgnl_f_skw = sgnl_stckf.skew(axis=1)
#         sgnl_f_kts = sgnl_stckf.kurt(axis=1)
#         sgnl_f_sem = sgnl_stckf.sem(axis=1)
#         sgnl_f_std = sgnl_stckf.std(axis=1)
#         sgnl_f_nuq = sgnl_stckf.nunique(axis=1)
        
#         sgnl_b_avg = sgnl_stckb.mean(1)
#         sgnl_b_q10 = sgnl_stckb.quantile(0.10, axis=1)
#         sgnl_b_q25 = sgnl_stckb.quantile(0.25, axis=1)
#         sgnl_b_q50 = sgnl_stckb.quantile(0.50, axis=1)
#         sgnl_b_q75 = sgnl_stckb.quantile(0.75, axis=1)
#         sgnl_b_q90 = sgnl_stckb.quantile(0.90, axis=1)
#         sgnl_b_skw = sgnl_stckb.skew(axis=1)
#         sgnl_b_kts = sgnl_stckb.kurt(axis=1)
#         sgnl_b_sem = sgnl_stckb.sem(axis=1)
#         sgnl_b_std = sgnl_stckb.std(axis=1)
#         sgnl_b_nuq = sgnl_stckb.nunique(axis=1)
        
#         sgnl_0_avg = sgnl_stck0.mean(1)
#         sgnl_0_q10 = sgnl_stck0.quantile(0.10, axis=1)
#         sgnl_0_q25 = sgnl_stck0.quantile(0.25, axis=1)
#         sgnl_0_q50 = sgnl_stck0.quantile(0.50, axis=1)
#         sgnl_0_q75 = sgnl_stck0.quantile(0.75, axis=1)
#         sgnl_0_q90 = sgnl_stck0.quantile(0.90, axis=1)
#         sgnl_0_skw = sgnl_stck0.skew(axis=1)
#         sgnl_0_kts = sgnl_stck0.kurt(axis=1)
#         sgnl_0_sem = sgnl_stck0.sem(axis=1)
#         sgnl_0_std = sgnl_stck0.std(axis=1)
#         sgnl_0_nuq = sgnl_stck0.nunique(axis=1)

#         sr_p1b5 = ts_utils.serie_stacker(sgnl_p1b5, step_size=step, block_size=wndw)
#         sr_p1b4 = ts_utils.serie_stacker(sgnl_p1b4, step_size=step, block_size=wndw)
        sr_p1b3 = ts_utils.serie_stacker(sgnl_p1b3, step_size=step, block_size=wndw)
        sr_p1b2 = ts_utils.serie_stacker(sgnl_p1b2, step_size=step, block_size=wndw)
        sr_p1b1 = ts_utils.serie_stacker(sgnl_p1b1, step_size=step, block_size=wndw)
        sr_p1  = ts_utils.serie_stacker(sgnl_p1, step_size=step, block_size=wndw)
        sr_p1f1 = ts_utils.serie_stacker(sgnl_p1f1, step_size=step, block_size=wndw)
        sr_p1f2 = ts_utils.serie_stacker(sgnl_p1f2, step_size=step, block_size=wndw)
        sr_p1f3 = ts_utils.serie_stacker(sgnl_p1f3, step_size=step, block_size=wndw)
#         sr_p1f4 = ts_utils.serie_stacker(sgnl_p1f4, step_size=step, block_size=wndw)
#         sr_p1f5 = ts_utils.serie_stacker(sgnl_p1f5, step_size=step, block_size=wndw)
        
#         sr_p1bx = ts_utils.serie_stacker(sgnl_p1bx, step_size=step, block_size=wndw)
#         sr_p1b9 = ts_utils.serie_stacker(sgnl_p1b9, step_size=step, block_size=wndw)
#         sr_p1b8 = ts_utils.serie_stacker(sgnl_p1b8, step_size=step, block_size=wndw)
#         sr_p1b7 = ts_utils.serie_stacker(sgnl_p1b7, step_size=step, block_size=wndw)
#         sr_p1b6 = ts_utils.serie_stacker(sgnl_p1b6, step_size=step, block_size=wndw)
#         sr_p1f6 = ts_utils.serie_stacker(sgnl_p1f6, step_size=step, block_size=wndw)
#         sr_p1f7 = ts_utils.serie_stacker(sgnl_p1f7, step_size=step, block_size=wndw)
#         sr_p1f8 = ts_utils.serie_stacker(sgnl_p1f8, step_size=step, block_size=wndw)
#         sr_p1f9 = ts_utils.serie_stacker(sgnl_p1f9, step_size=step, block_size=wndw)
#         sr_p1fx = ts_utils.serie_stacker(sgnl_p1fx, step_size=step, block_size=wndw)
        
#         sr_grad = ts_utils.serie_stacker(sgnl_grad, step_size=step, block_size=wndw)
        
#         sr_f_avg = ts_utils.serie_stacker(sgnl_f_avg, step_size=step, block_size=wndw)
#         sr_f_q10 = ts_utils.serie_stacker(sgnl_f_q10, step_size=step, block_size=wndw)
#         sr_f_q25 = ts_utils.serie_stacker(sgnl_f_q25, step_size=step, block_size=wndw)
#         sr_f_q50 = ts_utils.serie_stacker(sgnl_f_q50, step_size=step, block_size=wndw)
#         sr_f_q75 = ts_utils.serie_stacker(sgnl_f_q75, step_size=step, block_size=wndw)
#         sr_f_q90 = ts_utils.serie_stacker(sgnl_f_q90, step_size=step, block_size=wndw)
#         sr_f_skw = ts_utils.serie_stacker(sgnl_f_skw, step_size=step, block_size=wndw)
#         sr_f_kts = ts_utils.serie_stacker(sgnl_f_kts, step_size=step, block_size=wndw)
#         sr_f_sem = ts_utils.serie_stacker(sgnl_f_sem, step_size=step, block_size=wndw)
#         sr_f_std = ts_utils.serie_stacker(sgnl_f_std, step_size=step, block_size=wndw)
#         sr_f_nuq = ts_utils.serie_stacker(sgnl_f_nuq, step_size=step, block_size=wndw)
        
#         sr_b_avg = ts_utils.serie_stacker(sgnl_b_avg, step_size=step, block_size=wndw)
#         sr_b_q10 = ts_utils.serie_stacker(sgnl_b_q10, step_size=step, block_size=wndw)
#         sr_b_q25 = ts_utils.serie_stacker(sgnl_b_q25, step_size=step, block_size=wndw)
#         sr_b_q50 = ts_utils.serie_stacker(sgnl_b_q50, step_size=step, block_size=wndw)
#         sr_b_q75 = ts_utils.serie_stacker(sgnl_b_q75, step_size=step, block_size=wndw)
#         sr_b_q90 = ts_utils.serie_stacker(sgnl_b_q90, step_size=step, block_size=wndw)
#         sr_b_skw = ts_utils.serie_stacker(sgnl_b_skw, step_size=step, block_size=wndw)
#         sr_b_kts = ts_utils.serie_stacker(sgnl_b_kts, step_size=step, block_size=wndw)
#         sr_b_sem = ts_utils.serie_stacker(sgnl_b_sem, step_size=step, block_size=wndw)
#         sr_b_std = ts_utils.serie_stacker(sgnl_b_std, step_size=step, block_size=wndw)
#         sr_b_nuq = ts_utils.serie_stacker(sgnl_b_nuq, step_size=step, block_size=wndw)
        
#         sr_0_avg = ts_utils.serie_stacker(sgnl_0_avg, step_size=step, block_size=wndw)
#         sr_0_q10 = ts_utils.serie_stacker(sgnl_0_q10, step_size=step, block_size=wndw)
#         sr_0_q25 = ts_utils.serie_stacker(sgnl_0_q25, step_size=step, block_size=wndw)
#         sr_0_q50 = ts_utils.serie_stacker(sgnl_0_q50, step_size=step, block_size=wndw)
#         sr_0_q75 = ts_utils.serie_stacker(sgnl_0_q75, step_size=step, block_size=wndw)
#         sr_0_q90 = ts_utils.serie_stacker(sgnl_0_q90, step_size=step, block_size=wndw)
#         sr_0_skw = ts_utils.serie_stacker(sgnl_0_skw, step_size=step, block_size=wndw)
#         sr_0_kts = ts_utils.serie_stacker(sgnl_0_kts, step_size=step, block_size=wndw)
#         sr_0_sem = ts_utils.serie_stacker(sgnl_0_sem, step_size=step, block_size=wndw)
#         sr_0_std = ts_utils.serie_stacker(sgnl_0_std, step_size=step, block_size=wndw)
#         sr_0_nuq = ts_utils.serie_stacker(sgnl_0_nuq, step_size=step, block_size=wndw)

#         sr_diff_p1b5 = ts_utils.serie_stacker(sgnl_diff_p1b5, step_size=step, block_size=wndw)
#         sr_diff_p1b4 = ts_utils.serie_stacker(sgnl_diff_p1b4, step_size=step, block_size=wndw)
        sr_diff_p1b3 = ts_utils.serie_stacker(sgnl_diff_p1b3, step_size=step, block_size=wndw)
        sr_diff_p1b2 = ts_utils.serie_stacker(sgnl_diff_p1b2, step_size=step, block_size=wndw)
        sr_diff_p1b1 = ts_utils.serie_stacker(sgnl_diff_p1b1, step_size=step, block_size=wndw)
#         sr_diff_p1f5 = ts_utils.serie_stacker(sgnl_diff_p1f5, step_size=step, block_size=wndw)
#         sr_diff_p1f4 = ts_utils.serie_stacker(sgnl_diff_p1f4, step_size=step, block_size=wndw)
        sr_diff_p1f3 = ts_utils.serie_stacker(sgnl_diff_p1f3, step_size=step, block_size=wndw)
        sr_diff_p1f2 = ts_utils.serie_stacker(sgnl_diff_p1f2, step_size=step, block_size=wndw)
        sr_diff_p1f1 = ts_utils.serie_stacker(sgnl_diff_p1f1, step_size=step, block_size=wndw)

#         sr_diff_p1b5_p1f5 = ts_utils.serie_stacker(sgnl_diff_p1b5_p1f5, step_size=step, block_size=wndw)
        sr_diff_p1b3_p1f3 = ts_utils.serie_stacker(sgnl_diff_p1b3_p1f3, step_size=step, block_size=wndw)
        sr_diff_p1b1_p1f1 = ts_utils.serie_stacker(sgnl_diff_p1b1_p1f1, step_size=step, block_size=wndw)
        
#         sr_exp = ts_utils.serie_stacker(sgnl_exp, step_size=step, block_size=wndw)

#         sr_intgrl0 = ts_utils.serie_stacker(sgnl_intgrl0, step_size=step, block_size=wndw)
#         sr_intgrlf = ts_utils.serie_stacker(sgnl_intgrlf, step_size=step, block_size=wndw)
#         sr_intgrlb = ts_utils.serie_stacker(sgnl_intgrlb, step_size=step, block_size=wndw)

        sr_p1_b3f3_b2f2_b1f1  = ts_utils.serie_stacker(
            sgnl_p1b3*sgnl_p1f3+sgnl_p1b2*sgnl_p1f2+sgnl_p1b1*sgnl_p1f1,
            step_size=step,
            block_size=wndw
        )

        sr_p1_b3f1_b2f2_b1f3  = ts_utils.serie_stacker(
            sgnl_p1b3*sgnl_p1f1-sgnl_p1b2*sgnl_p1f2+sgnl_p1b1*sgnl_p1f3,
            step_size=step,
            block_size=wndw
        )

        # ------------------------------------------------
        sgnl_p2 = sgnl_p1**2
        p2_fill = sgnl_p2.mean()

#         sgnl_p2b5 = sgnl_p2.shift(-5, fill_value=p2_fill)
#         sgnl_p2b4 = sgnl_p2.shift(-4, fill_value=p2_fill)
        sgnl_p2b3 = sgnl_p2.shift(-3, fill_value=p2_fill)
        sgnl_p2b2 = sgnl_p2.shift(-2, fill_value=p2_fill)
        sgnl_p2b1 = sgnl_p2.shift(-1, fill_value=p2_fill)
        sgnl_p2f1 = sgnl_p2.shift(1, fill_value=p2_fill)
        sgnl_p2f2 = sgnl_p2.shift(2, fill_value=p2_fill)
        sgnl_p2f3 = sgnl_p2.shift(3, fill_value=p2_fill)
#         sgnl_p2f4 = sgnl_p2.shift(4, fill_value=p2_fill)
#         sgnl_p2f5 = sgnl_p2.shift(5, fill_value=p2_fill)

#         sgnl_diff_p2b5 = sgnl_p2b5-sgnl_p2
#         sgnl_diff_p2b4 = sgnl_p2b4-sgnl_p2
        sgnl_diff_p2b3 = sgnl_p2b3-sgnl_p2
        sgnl_diff_p2b2 = sgnl_p2b2-sgnl_p2
        sgnl_diff_p2b1 = sgnl_p2b1-sgnl_p2
#         sgnl_diff_p2f5 = sgnl_p2f5-sgnl_p2
#         sgnl_diff_p2f4 = sgnl_p2f4-sgnl_p2
        sgnl_diff_p2f3 = sgnl_p2f3-sgnl_p2
        sgnl_diff_p2f2 = sgnl_p2f2-sgnl_p2
        sgnl_diff_p2f1 = sgnl_p2f1-sgnl_p2

#         sr_p2b5 = ts_utils.serie_stacker(sgnl_p2b5, step_size=step, block_size=wndw)
#         sr_p2b4 = ts_utils.serie_stacker(sgnl_p2b4, step_size=step, block_size=wndw)
        sr_p2b3 = ts_utils.serie_stacker(sgnl_p2b3, step_size=step, block_size=wndw)
        sr_p2b2 = ts_utils.serie_stacker(sgnl_p2b2, step_size=step, block_size=wndw)
        sr_p2b1 = ts_utils.serie_stacker(sgnl_p2b1, step_size=step, block_size=wndw)
        sr_p2  = ts_utils.serie_stacker(sgnl_p2, step_size=step, block_size=wndw)
        sr_p2f1 = ts_utils.serie_stacker(sgnl_p2f1, step_size=step, block_size=wndw)
        sr_p2f2 = ts_utils.serie_stacker(sgnl_p2f2, step_size=step, block_size=wndw)
        sr_p2f3 = ts_utils.serie_stacker(sgnl_p2f3, step_size=step, block_size=wndw)
#         sr_p2f4 = ts_utils.serie_stacker(sgnl_p2f4, step_size=step, block_size=wndw)
#         sr_p2f5 = ts_utils.serie_stacker(sgnl_p2f5, step_size=step, block_size=wndw)

#         sr_diff_p2b5 = ts_utils.serie_stacker(sgnl_diff_p2b5, step_size=step, block_size=wndw)
#         sr_diff_p2b4 = ts_utils.serie_stacker(sgnl_diff_p2b4, step_size=step, block_size=wndw)
        sr_diff_p2b3 = ts_utils.serie_stacker(sgnl_diff_p2b3, step_size=step, block_size=wndw)
        sr_diff_p2b2 = ts_utils.serie_stacker(sgnl_diff_p2b2, step_size=step, block_size=wndw)
        sr_diff_p2b1 = ts_utils.serie_stacker(sgnl_diff_p2b1, step_size=step, block_size=wndw)
#         sr_diff_p2f5 = ts_utils.serie_stacker(sgnl_diff_p2f5, step_size=step, block_size=wndw)
#         sr_diff_p2f4 = ts_utils.serie_stacker(sgnl_diff_p2f4, step_size=step, block_size=wndw)
        sr_diff_p2f3 = ts_utils.serie_stacker(sgnl_diff_p2f3, step_size=step, block_size=wndw)
        sr_diff_p2f2 = ts_utils.serie_stacker(sgnl_diff_p2f2, step_size=step, block_size=wndw)
        sr_diff_p2f1 = ts_utils.serie_stacker(sgnl_diff_p2f1, step_size=step, block_size=wndw)

        sr = np.stack(
            [
                sr_p1, 
                sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, 
                #sr_p1b5, sr_p1b4, sr_p1f4, sr_p1f5,
                #sr_p1b6, sr_p1b7, sr_p1b8, sr_p1b9, sr_p1bx, sr_p1f6, sr_p1f7, sr_p1f8, sr_p1f9, sr_p1fx,
                #sr_f_avg, sr_f_q10, sr_f_q25, sr_f_q50, sr_f_q75, sr_f_q90, sr_f_skw, sr_f_kts, sr_f_sem, sr_f_std, sr_f_nuq,
                #sr_b_avg, sr_b_q10, sr_b_q25, sr_b_q50, sr_b_q75, sr_b_q90, sr_b_skw, sr_b_kts, sr_b_sem, sr_b_std, sr_b_nuq,
                #sr_0_avg, sr_0_q10, sr_0_q25, sr_0_q50, sr_0_q75, sr_0_q90, sr_0_skw, sr_0_kts, sr_0_sem, sr_0_std, sr_0_nuq,
                sr_diff_p1b3, sr_diff_p1b2, sr_diff_p1b1, sr_diff_p1f3, sr_diff_p1f2, sr_diff_p1f1,
                #sr_diff_p1b5, sr_diff_p1b4, sr_diff_p1f5, sr_diff_p1f4, 
                #sr_diff_p1b5_p1f5, 
                sr_diff_p1b3_p1f3, sr_diff_p1b1_p1f1,
                #sr_grad,
                #sr_exp,
                #sr_intgrl0, sr_intgrlf, sr_intgrlb,
                sr_p1_b3f3_b2f2_b1f1, sr_p1_b3f1_b2f2_b1f3,
                sr_p2, 
                sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, 
                #sr_p2b5, sr_p2b4, sr_p2f4, sr_p2f5,
                sr_diff_p2b3, sr_diff_p2b2, sr_diff_p2b1, sr_diff_p2f3, sr_diff_p2f2, sr_diff_p2f1,
                #sr_diff_p2b5, sr_diff_p2b4, sr_diff_p2f5, sr_diff_p2f4, 
            ],
            axis=2
        )
        
        if mode == 'train':
            tr = ts_utils.serie_stacker(trgt, step_size=step, block_size=wndw)
            bp.pack_ndarray_to_file(sr, '../input/feats_srs/trn_srs_dat_g{:d}_s{:d}_w{:d}_feat_{}.bp'.format(i, step, wndw, tag))
            bp.pack_ndarray_to_file(tr, '../input/feats_srs/trn_srs_lbl_g{:d}_s{:d}_w{:d}_feat_{}.bp'.format(i, step, wndw, tag))
        else:
            bp.pack_ndarray_to_file(sr, '../input/feats_srs/tst_srs_dat_g{:d}_s{:d}_w{:d}_feat_{}.bp'.format(i, step, wndw, tag))

        del sgnl_ndcs

        del sgnl_p1
        del sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, sgnl_p1f1, sgnl_p1f2, sgnl_p1f3
        #del sgnl_p1b5, sgnl_p1b4, sgnl_p1f4, sgnl_p1f5
        #del sgnl_p1b6, sgnl_p1b7, sgnl_p1b8, sgnl_p1b9, sgnl_p1bx, sgnl_p1f6, sgnl_p1f7, sgnl_p1f8, sgnl_p1f9, sgnl_p1fx
        #del sgnl_diff_p1b5, sgnl_diff_p1b4
        del sgnl_diff_p1b3, sgnl_diff_p1b2, sgnl_diff_p1b1
        #del sgnl_diff_p1f5, sgnl_diff_p1f4, 
        del sgnl_diff_p1f3, sgnl_diff_p1f2, sgnl_diff_p1f1
        #del sgnl_diff_p1b5_p1f5, 
        del sgnl_diff_p1b3_p1f3, sgnl_diff_p1b1_p1f1
        #del sgnl_grad
        #del sgnl_f_avg, sgnl_f_q10, sgnl_f_q25, sgnl_f_q50, sgnl_f_q75, sgnl_f_q90, sgnl_f_skw, sgnl_f_kts, sgnl_f_sem, sgnl_f_std, sgnl_f_nuq
        #del sgnl_b_avg, sgnl_b_q10, sgnl_b_q25, sgnl_b_q50, sgnl_b_q75, sgnl_b_q90, sgnl_b_skw, sgnl_b_kts, sgnl_b_sem, sgnl_b_std, sgnl_b_nuq
        #del sgnl_0_avg, sgnl_0_q10, sgnl_0_q25, sgnl_0_q50, sgnl_0_q75, sgnl_0_q90, sgnl_0_skw, sgnl_0_kts, sgnl_0_sem, sgnl_0_std, sgnl_0_nuq
        #del sgnl_exp
        #del sgnl_stck0, sgnl_stckf, sgnl_stckb
        #del sgnl_intgrl0, sgnl_intgrlf, sgnl_intgrlb

        del sgnl_p2
        del sgnl_p2b3, sgnl_p2b2, sgnl_p2b1, sgnl_p2f1, sgnl_p2f2, sgnl_p2f3
        #del sgnl_p2b5, sgnl_p2b4, sgnl_p2f4, sgnl_p2f5
        del sgnl_diff_p2b3, sgnl_diff_p2b2, sgnl_diff_p2b1
        #del sgnl_diff_p2b5, sgnl_diff_p2b4
        del sgnl_diff_p2f3, sgnl_diff_p2f2, sgnl_diff_p2f1
        #del sgnl_diff_p2f5, sgnl_diff_p2f4

        del sr_p1
        del sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3
        #del sr_p1b5, sr_p1b4, sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, sr_p1f4, sr_p1f5
        #del sr_p1bx, sr_p1b9, sr_p1b8, sr_p1b7, sr_p1b6, sr_p1f6, sr_p1f7, sr_p1f8, sr_p1f9, sr_p1fx
        #del sr_diff_p1b5, sr_diff_p1b4, sr_diff_p1f5, sr_diff_p1f4, 
        del sr_diff_p1b3, sr_diff_p1b2, sr_diff_p1b1, sr_diff_p1f3, sr_diff_p1f2, sr_diff_p1f1
        #del sr_diff_p1b5_p1f5, 
        #del sr_diff_p1b3_p1f3, sr_diff_p1b1_p1f1
        #del sr_grad
        #del sr_f_avg, sr_f_q10, sr_f_q25, sr_f_q50, sr_f_q75, sr_f_q90, sr_f_skw, sr_f_kts, sr_f_sem, sr_f_std, sr_f_nuq,
        #del sr_b_avg, sr_b_q10, sr_b_q25, sr_b_q50, sr_b_q75, sr_b_q90, sr_b_skw, sr_b_kts, sr_b_sem, sr_b_std, sr_b_nuq,
        #del sr_0_avg, sr_0_q10, sr_0_q25, sr_0_q50, sr_0_q75, sr_0_q90, sr_0_skw, sr_0_kts, sr_0_sem, sr_0_std, sr_0_nuq,
        #del sr_exp
        #del sr_intgrl0, sr_intgrlf, sr_intgrlb
        del sr_p1_b3f3_b2f2_b1f1, sr_p1_b3f1_b2f2_b1f3

        del sr_p2
        del sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3
        #del sr_p2b5, sr_p2b4, sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, sr_p2f4, sr_p2f5
        #del sr_diff_p2b5, sr_diff_p2b4, sr_diff_p2f5, sr_diff_p2f4, 
        del sr_diff_p2b3, sr_diff_p2b2, sr_diff_p2b1, sr_diff_p2f3, sr_diff_p2f2, sr_diff_p2f1

        del sr
        
        if mode == 'train':
            del trgt, tr